# Transformer (3 multimodal input)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# ===== 参数 =====
path = "merged.parquet"
window_size = 7
train_ratio = 0.8
batch_size = 64

# ===== 1️⃣ 读取 parquet =====
print("📥 读取数据中...")
df = pd.read_parquet(path)
df = df.sort_values(["StockCode", "Date"]).reset_index(drop=True)
print(f"✅ 共 {len(df)} 行, {df.shape[1]} 列")

# ===== 2️⃣ 特征分类 =====
emotion_cols = [
    'Emotion_Index_closing', 'Emotion_Index_trading',
    'Total_Posts_closing', 'Total_Posts_trading',
    'Total_Click_Count_closing', 'Total_Click_Count_trading',
    'Emotion_Momentum_3d_closing', 'Emotion_Momentum_3d_trading',
    'Emotion_Momentum_5d_closing', 'Emotion_Momentum_5d_trading'
]
stock_cols = ['log_return', 'amplitude', 'rsi_14', 'macd', 'bb_width_20', 'obv']
text_cols = [f"dim_{i}_{t}" for i in range(1024) for t in ("closing", "trading")]

feature_cols = emotion_cols + stock_cols + text_cols
print(f"🧩 特征总数: {len(feature_cols)}")

# ===== 3️⃣ 构造所有窗口样本 =====
samples = []

for code, group in df.groupby("StockCode"):
    group = group.reset_index(drop=True)
    if len(group) <= window_size:
        continue

    emo = torch.tensor(group[emotion_cols].values, dtype=torch.float32)
    stk = torch.tensor(group[stock_cols].values, dtype=torch.float32)
    txt = torch.tensor(group[text_cols].values, dtype=torch.float32)
    label = torch.tensor(group["Label"].values, dtype=torch.int64)
    date = group["Date"].values

    for i in range(len(group) - window_size):
        samples.append({
            "StockCode": code,
            "end_date": date[i + window_size],
            "emotion": emo[i:i + window_size],
            "stock": stk[i:i + window_size],
            "text": txt[i:i + window_size],
            "label": label[i + window_size]
        })

print(f"✅ 共生成 {len(samples)} 个窗口样本")

# # ===== 4️⃣ 转换为 DataFrame（仅用于时间切分） =====
# dates = [s["end_date"] for s in samples]
# dates = sorted(pd.Series(dates).unique())
# split_date = dates[int(len(dates) * train_ratio)]
# print(f"📆 按时间划分：训练集截止到 {split_date.date()}")

# # ===== 5️⃣ 按时间划分训练/验证集 =====
# train_samples = [s for s in samples if s["end_date"] <= split_date]
# val_samples = [s for s in samples if s["end_date"] > split_date]
# print(f"📊 训练集 {len(train_samples)}，验证集 {len(val_samples)}")

# ===== 4️⃣ 警告：使用随机划分（仅用于学术实验，非正确做法） =====
from sklearn.model_selection import train_test_split

print("⚠️ 警告：正在使用随机划分，这会导致数据泄露和虚高的验证分数！")
train_samples, val_samples = train_test_split(samples, test_size=0.1, random_state=42)
print(f"📊 训练集 {len(train_samples)}，验证集 {len(val_samples)}")

# ===== 6️⃣ 定义 Dataset =====
class MultiFeatureDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        return {
            "emotion": s["emotion"],
            "stock": s["stock"],
            "text": s["text"],
            "label": s["label"]
        }

train_set = MultiFeatureDataset(train_samples)
val_set = MultiFeatureDataset(val_samples)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

print("✅ DataLoader 构建完成")

# ===== 7️⃣ 打印维度检查 =====
for batch in train_loader:
    print("emotion:", batch["emotion"].shape)  # [B, 30, 10]
    print("stock:", batch["stock"].shape)      # [B, 30, 6]
    print("text:", batch["text"].shape)        # [B, 30, 2048]
    print("label:", batch["label"].shape)      # [B]
    break


📥 读取数据中...
✅ 共 55381 行, 2073 列
🧩 特征总数: 2064
✅ 共生成 53771 个窗口样本
⚠️ 警告：正在使用随机划分，这会导致数据泄露和虚高的验证分数！
📊 训练集 48393，验证集 5378
✅ DataLoader 构建完成
emotion: torch.Size([64, 7, 10])
stock: torch.Size([64, 7, 6])
text: torch.Size([64, 7, 2048])
label: torch.Size([64])


In [ ]:
import torch
import torch.nn as nn

class PositionalEncoding(nn.Module):
    def __init__(self, dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-torch.log(torch.tensor(10000.0)) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # shape (1, max_len, dim)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1)]
        return x

class MultiModalTransformer(nn.Module):
    """
    - stock/emo/text -> 统一降到 proj_dim（默认128），再融合到 hidden_dim（默认512）
    - 加可学习位置编码
    - 用 TransformerEncoder 做时间建模
    - 用注意力池化(learnable time attention) 代替简单 mean 池化
    """
    def __init__(
        self,
        stock_dim: int = 6,
        emo_dim: int = 10,
        text_dim: int = 2048,
        proj_dim: int = 128,
        hidden_dim: int = 256,
        num_heads: int = 8,
        num_layers: int = 3,
        num_classes: int = 3,
        max_len: int = 512,
        dropout: float = 0.2,
    ):
        super().__init__()

        # === 各模态编码 ===
        # stock：直接线性映射到 proj_dim
        self.stock_proj = nn.Sequential(
            nn.LayerNorm(stock_dim),
            nn.Linear(stock_dim, proj_dim),
            nn.ReLU(),
            nn.LayerNorm(proj_dim)
        )

        # emotion：先做LayerNorm稳定数值，再映射
        self.emo_proj = nn.Sequential(
            nn.LayerNorm(emo_dim),
            nn.Linear(emo_dim, proj_dim),
            nn.ReLU(),
            nn.LayerNorm(proj_dim)
        )

        # text：强力降维 2048 -> 256 -> proj_dim，并做LayerNorm
        self.text_reduce = nn.Sequential(
            nn.Linear(text_dim, 256),
            nn.ReLU(),
            nn.Linear(256, proj_dim),
            nn.LayerNorm(proj_dim)
        )

        # === 融合到 hidden_dim（带轻微残差思路：先concat再线性） ===
        self.fuse = nn.Sequential(
            nn.Linear(proj_dim * 3, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.LayerNorm(hidden_dim)
        )

        self.pos_encoder = PositionalEncoding(
            max_len=max_len,
            dim=hidden_dim
        )

        # === Transformer 编码器 ===
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            activation="relu",
            dropout=dropout,
            batch_first=True,
            norm_first=True,   # 更稳定
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # === 时间注意力池化（代替 mean）===
        self.time_attn = nn.Sequential(
            nn.Linear(hidden_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

        # === 分类头 ===
        self.cls_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, stock_seq, emo_seq, text_seq):
        """
        输入：
            stock_seq: (B, T, stock_dim)
            emo_seq:   (B, T, emo_dim)
            text_seq:  (B, T, text_dim)
        输出：
            logits:    (B, num_classes)
        """
        # 1) 各模态特征到同一尺度
        s = self.stock_proj(stock_seq)          # (B, T, proj_dim)
        e = self.emo_proj(emo_seq)              # (B, T, proj_dim)
        t = self.text_reduce(text_seq)          # (B, T, proj_dim)

        # 2) 融合到 hidden_dim
        x = torch.cat([s, e, t], dim=-1)        # (B, T, 3*proj_dim)
        x = self.fuse(x)                        # (B, T, hidden_dim)

        # 3) 位置编码
        # T = x.size(1)
        # x = x + self.pos_embedding[:, :T, :]    # (B, T, hidden_dim)
        x = self.pos_encoder(x) # <--- 修改在这里

        # 4) 时间依赖编码
        enc_out = self.encoder(x)               # (B, T, hidden_dim)

        # 5) 注意力池化（学会关注关键时间步）
        attn = self.time_attn(enc_out)          # (B, T, 1)
        attn = torch.softmax(attn, dim=1)       # (B, T, 1)
        pooled = torch.sum(enc_out * attn, dim=1)  # (B, hidden_dim)

        # 6) 分类
        logits = self.cls_head(pooled)          # (B, num_classes)
        return logits



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

# ======= 1️⃣ 超参数配置 =======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 3
lr = 1e-4
num_epochs = 100
save_path = "best_model.pt"

from collections import Counter
import numpy as np

print("⚖️ 计算类别权重...")
# 1. 统计训练集中每个类别的样本数
train_labels = [s['label'].item() for s in train_samples]
label_counts = Counter(sorted(train_labels))

# 2. 计算权重 (使用标准的反比频率公式)
#    公式: weight[c] = total_samples / (num_classes * count[c])
total_samples = len(train_labels)
weights = [total_samples / (len(label_counts) * label_counts[i]) for i in sorted(label_counts.keys())]
class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

print(f"类别计数: {label_counts}")
print(f"计算出的类别权重: {class_weights}")

# ======= 2️⃣ 模型实例化 =======
model = MultiModalTransformer(
    stock_dim=6,
    emo_dim=10,
    text_dim=2048,
    proj_dim=128,
    hidden_dim=128,
    num_heads=4,
    num_layers=2,
    num_classes=num_classes,
    max_len=512,
    dropout=0.5
).to(device)

# ======= 3️⃣ 优化器 + 损失函数 =======
criterion = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# ======= 4️⃣ 训练与验证循环 =======
best_val_acc = 0.0

for epoch in range(1, num_epochs + 1):
    # print(f"\n🟢 Epoch {epoch}/{num_epochs}")
    # === Train ===
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for batch in train_loader:
        emo = batch["emotion"].to(device)
        stk = batch["stock"].to(device)
        txt = batch["text"].to(device)
        y = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(stk, emo, txt)
        loss = criterion(logits, y)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        train_loss += loss.item() * y.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    train_acc = correct / total
    train_loss /= total
    # scheduler.step()

    # === Validate ===
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    all_val_preds = []
    all_val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            emo = batch["emotion"].to(device)
            stk = batch["stock"].to(device)
            txt = batch["text"].to(device)
            y = batch["label"].to(device)

            logits = model(stk, emo, txt)
            loss = criterion(logits, y)

            val_loss += loss.item() * y.size(0)
            pred = logits.argmax(dim=1)
            val_correct += (pred == y).sum().item()
            val_total += y.size(0)

            all_val_preds.extend(pred.cpu().numpy())
            all_val_labels.extend(y.cpu().numpy())

    val_acc = val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch}: Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%", end = "   ")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc*100:.2f}%")

    cm = confusion_matrix(all_val_labels, all_val_preds)
    print("Validation Confusion Matrix:")
    print(cm)

    # === Save best model ===
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved ({best_val_acc*100:.2f}%)")

print(f"\n🏁 Training Finished! Best Val Acc = {best_val_acc*100:.2f}%")


⚖️ 计算类别权重...
类别计数: Counter({1: 34224, 2: 7317, 0: 6852})
计算出的类别权重: tensor([2.3542, 0.4713, 2.2046], device='cuda:0')


/home/msai/liny0145/.conda/envs/py312/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Epoch 1: Train Loss: 1.1745 | Train Acc: 18.19%   Val Loss:   1.1570 | Val Acc:   15.71%
Validation Confusion Matrix:
[[ 435    0  364]
 [1342    1 2439]
 [ 388    0  409]]
✅ Best model saved (15.71%)
Epoch 2: Train Loss: 1.1605 | Train Acc: 33.02%   Val Loss:   1.1473 | Val Acc:   46.17%
Validation Confusion Matrix:
[[ 443  227  129]
 [1215 1887  680]
 [ 418  226  153]]
✅ Best model saved (46.17%)
Epoch 3: Train Loss: 1.1559 | Train Acc: 39.37%   Val Loss:   1.1452 | Val Acc:   44.81%
Validation Confusion Matrix:
[[ 503  181  115]
 [1400 1773  609]
 [ 458  205  134]]
Epoch 4: Train Loss: 1.1524 | Train Acc: 41.86%   Val Loss:   1.1452 | Val Acc:   52.21%
Validation Confusion Matrix:
[[ 274  258  267]
 [ 625 2276  881]
 [ 257  282  258]]
✅ Best model saved (52.21%)
Epoch 5: Train Loss: 1.1501 | Train Acc: 43.02%   Val Loss:   1.1499 | Val Acc:   36.95%
Validation Confusion Matrix:
[[ 503  103  193]
 [1340 1278 1164]
 [ 476  115  206]]
Epoch 6: Train Loss: 1.1496 | Train Acc: 43.54%   V

# Transformer (2 multimodal input)

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# ===== 参数 =====
path = "merged.parquet"
window_size = 10
test_ratio = 0.2
batch_size = 64

# ===== 1️⃣ 读取 parquet =====
print("📥 读取数据中...")
df = pd.read_parquet(path)
df = df.sort_values(["StockCode", "Date"]).reset_index(drop=True)
print(f"✅ 共 {len(df)} 行, {df.shape[1]} 列")

# ===== 2️⃣ 特征分类 (不使用 text_cols) =====
emotion_cols = ['Emotion_Index_closing',
 'Emotion_Index_trading',
 'Emotion_Momentum_3d_closing',
 'Emotion_Momentum_3d_trading']
stock_cols = ['log_return', 'amplitude']

feature_cols = emotion_cols + stock_cols
print(f"🧩 特征总数: {len(feature_cols)}")

# ===== 3️⃣ 构造所有窗口样本 =====
samples = []

for code, group in df.groupby("StockCode"):
    group = group.reset_index(drop=True)
    if len(group) <= window_size:
        continue

    # 只提取 emotion 和 stock 特征
    emo = torch.tensor(group[emotion_cols].values, dtype=torch.float32)
    stk = torch.tensor(group[stock_cols].values, dtype=torch.float32)
    label = torch.tensor(group["Label"].values, dtype=torch.int64)
    date = group["Date"].values

    for i in range(len(group) - window_size):
        samples.append({
            "StockCode": code,
            "end_date": date[i + window_size],
            "emotion": emo[i:i + window_size],
            "stock": stk[i:i + window_size],
            "label": label[i + window_size]
        })

print(f"✅ 共生成 {len(samples)} 个窗口样本")

# ===== 4️⃣ 随机划分训练/验证集 (根据您的要求) =====
# 警告：对于时序数据，随机划分可能导致数据泄露和评估结果虚高。
print("⚠️ 警告：正在使用随机划分。")
train_samples, val_samples = train_test_split(samples, test_size=test_ratio, random_state=42, shuffle=True)
print(f"📊 训练集 {len(train_samples)}，验证集 {len(val_samples)}")

# ===== 5️⃣ 定义 Dataset (新名称: StockEmotionDataset) =====
class StockEmotionDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        s = self.samples[idx]
        return {
            "emotion": s["emotion"],
            "stock": s["stock"],
            "label": s["label"]
        }

train_set = StockEmotionDataset(train_samples)
val_set = StockEmotionDataset(val_samples)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, pin_memory=True)

print("✅ DataLoader 构建完成")

# ===== 6️⃣ 打印维度检查 =====
for batch in train_loader:
    print("emotion:", batch["emotion"].shape)
    print("stock:", batch["stock"].shape)
    print("label:", batch["label"].shape)
    break

📥 读取数据中...
✅ 共 55381 行, 2073 列
🧩 特征总数: 6
✅ 共生成 53081 个窗口样本
⚠️ 警告：正在使用随机划分。
📊 训练集 42464，验证集 10617
✅ DataLoader 构建完成
emotion: torch.Size([64, 10, 4])
stock: torch.Size([64, 10, 2])
label: torch.Size([64])


In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    """
    固定的正弦/余弦位置编码模块。
    """
    def __init__(self, dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # Shape: (1, max_len, dim)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: (Batch, SeqLen, Dim)
        """
        x = x + self.pe[:, :x.size(1)]
        return x

class TwoModalTransformer(nn.Module):
    """
    一个更精简的 Transformer 模型，用于处理 stock 和 emotion 两种模态数据。
    - 使用固定的正余弦位置编码。
    """
    def __init__(
        self,
        stock_dim: int = 6,
        emo_dim: int = 10,
        proj_dim: int = 64,
        hidden_dim: int = 256,
        num_heads: int = 4,
        num_layers: int = 2,
        num_classes: int = 3,
        max_len: int = 512,
        dropout: float = 0.2,
    ):
        super().__init__()

        # === 各模态编码 ===
        self.stock_proj = nn.Sequential(
            nn.LayerNorm(stock_dim),
            nn.Linear(stock_dim, proj_dim),
            nn.ReLU(),
            nn.LayerNorm(proj_dim)
        )

        self.emo_proj = nn.Sequential(
            nn.LayerNorm(emo_dim),
            nn.Linear(emo_dim, proj_dim),
            nn.ReLU(),
            nn.LayerNorm(proj_dim)
        )

        # === 融合到 hidden_dim ===
        self.fuse = nn.Sequential(
            nn.Linear(proj_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.LayerNorm(hidden_dim)
        )

        # === 位置编码 (使用固定的正余弦编码) ===
        self.pos_encoder = PositionalEncoding(dim=hidden_dim, max_len=max_len)

        # === Transformer 编码器 ===
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            activation="relu",
            dropout=dropout,
            batch_first=True,
            norm_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # === 时间注意力池化 ===
        self.time_attn = nn.Sequential(
            nn.Linear(hidden_dim, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
        )

        # === 分类头 ===
        self.cls_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, num_classes)
        )

    def forward(self, stock_seq, emo_seq):
        """
        输入：
            stock_seq: (B, T, stock_dim)
            emo_seq:   (B, T, emo_dim)
        输出：
            logits:    (B, num_classes)
        """
        # 1) 各模态特征到同一尺度
        s = self.stock_proj(stock_seq) # (B, T, proj_dim)
        e = self.emo_proj(emo_seq)     # (B, T, proj_dim)

        # 2) 融合
        x = torch.cat([s, e], dim=-1)   # (B, T, 2*proj_dim)
        x = self.fuse(x)               # (B, T, hidden_dim)

        # 3) 位置编码
        x = self.pos_encoder(x) # <--- 修改在这里

        # 4) 时间依赖编码
        enc_out = self.encoder(x)

        # 5) 注意力池化
        attn = self.time_attn(enc_out)
        attn = torch.softmax(attn, dim=1)
        pooled = torch.sum(enc_out * attn, dim=1)

        # 6) 分类
        logits = self.cls_head(pooled)
        return logits

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

# ======= 1️⃣ 超参数配置 =======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

num_classes = 3
lr = 3e-5             # 建议使用稍低的学习率
num_epochs = 100
save_path = "best_model_2_modal.pt"

from collections import Counter
import numpy as np

print("⚖️ 计算类别权重...")
# 1. 统计训练集中每个类别的样本数
train_labels = [s['label'].item() for s in train_samples]
label_counts = Counter(sorted(train_labels))

# 2. 计算权重 (使用标准的反比频率公式)
#    公式: weight[c] = total_samples / (num_classes * count[c])
total_samples = len(train_labels)
weights = [total_samples / (len(label_counts) * label_counts[i]) for i in sorted(label_counts.keys())]
class_weights = torch.tensor(weights, dtype=torch.float32).to(device)

print(f"类别计数: {label_counts}")
print(f"计算出的类别权重: {class_weights}")

# ======= 2️⃣ 模型实例化 =======
# 使用新的 TwoModalTransformer 并传入建议的、更抗过拟合的超参数
model = TwoModalTransformer(
    stock_dim=len(stock_cols),
    emo_dim=len(emotion_cols),
    proj_dim=64,
    hidden_dim=256,
    num_heads=4,
    num_layers=2,
    num_classes=num_classes,
    max_len=window_size + 1, # 设置为略大于窗口长度
    dropout=0.1              # 使用更强的 dropout
).to(device)

# ======= 3️⃣ 优化器 + 损失函数 =======
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4) # 使用更强的 weight_decay
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

# ======= 4️⃣ 训练与验证循环 =======
best_val_acc = 0.0

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss, correct, total = 0.0, 0, 0

    for batch in train_loader:
        emo = batch["emotion"].to(device)
        stk = batch["stock"].to(device)
        y = batch["label"].to(device)

        optimizer.zero_grad()
        logits = model(stk, emo) # 只传入 stock 和 emotion
        loss = criterion(logits, y)

        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        train_loss += loss.item() * y.size(0)
        pred = logits.argmax(dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    train_acc = correct / total
    train_loss /= total
    # scheduler.step()

    # === Validate ===
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    all_val_preds = []
    all_val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            emo = batch["emotion"].to(device)
            stk = batch["stock"].to(device)
            y = batch["label"].to(device)

            logits = model(stk, emo)
            loss = criterion(logits, y)

            val_loss += loss.item() * y.size(0)
            pred = logits.argmax(dim=1)
            val_correct += (pred == y).sum().item()
            val_total += y.size(0)

            all_val_preds.extend(pred.cpu().numpy())
            all_val_labels.extend(y.cpu().numpy())

    val_acc = val_correct / val_total
    val_loss /= val_total

    print(f"Epoch {epoch}: Train Loss: {train_loss:.4f} | Train Acc: {train_acc*100:.2f}%", end = "   ")
    print(f"Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc*100:.2f}%")

    cm = confusion_matrix(all_val_labels, all_val_preds)
    print("Validation Confusion Matrix:")
    print(cm)

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), save_path)
        print(f"✅ Best model saved with Val Acc: {best_val_acc:.4f}")

print(f"\n🏁 Training Finished! Best Val Acc = {best_val_acc:.4f}")

Using device: cuda
⚖️ 计算类别权重...
类别计数: Counter({0: 16649, 2: 15614, 1: 10201})
计算出的类别权重: tensor([0.8502, 1.3876, 0.9065], device='cuda:0')


/home/msai/liny0145/.conda/envs/py312/lib/python3.12/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


Epoch 1: Train Loss: 1.1002 | Train Acc: 35.20%   Val Loss:   1.0960 | Val Acc:   31.34%
Validation Confusion Matrix:
[[1261 2667  336]
 [ 598 1767  163]
 [1089 2437  299]]
✅ Best model saved with Val Acc: 0.3134
Epoch 2: Train Loss: 1.0955 | Train Acc: 34.86%   Val Loss:   1.0940 | Val Acc:   33.77%
Validation Confusion Matrix:
[[1762 2124  378]
 [ 850 1484  194]
 [1531 1955  339]]
✅ Best model saved with Val Acc: 0.3377
Epoch 3: Train Loss: 1.0934 | Train Acc: 35.21%   Val Loss:   1.0914 | Val Acc:   35.92%
Validation Confusion Matrix:
[[1881 1871  512]
 [ 845 1369  314]
 [1575 1686  564]]
✅ Best model saved with Val Acc: 0.3592
Epoch 4: Train Loss: 1.0917 | Train Acc: 36.01%   Val Loss:   1.0931 | Val Acc:   32.80%
Validation Confusion Matrix:
[[1492 2514  258]
 [ 655 1743  130]
 [1298 2280  247]]
Epoch 5: Train Loss: 1.0910 | Train Acc: 36.12%   Val Loss:   1.0914 | Val Acc:   35.07%
Validation Confusion Matrix:
[[1596 1836  832]
 [ 736 1360  432]
 [1377 1681  767]]
Epoch 6: Train 

KeyboardInterrupt: 

# 1 day, lgbm

## direct run

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

# 假设您的数据在一个名为 all_in_one_file 的 DataFrame 中
all_in_one_file = pd.read_parquet('merged.parquet')

# 选择您指定的特征列
emotion_cols = [
    'Emotion_Index_closing',
    'Emotion_Index_trading',
    'Emotion_Momentum_3d_closing',
    'Emotion_Momentum_3d_trading'
]
stock_cols = ['log_return', 'amplitude']

feature_cols = emotion_cols + stock_cols
# feature_cols = list(set(all_in_one_file.columns.tolist()) - set(['Label', 'Date', 'StockCode']))
target_col = 'Label'

# 确保所有特征列和目标列都在 DataFrame 中
required_cols = feature_cols + [target_col]
if not all(col in all_in_one_file.columns for col in required_cols):
    raise ValueError("DataFrame 中缺少所需的列")

# 准备特征和标签
X = all_in_one_file[feature_cols]
y = all_in_one_file[target_col]

# 划分训练集和测试集
# 为了保持时间序列的顺序，我们不随机打乱数据
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# --- 参数设置 ---
# 定义 LightGBM 的参数字典
# 您可以根据需要调整这些值
params = {
    'objective': 'multiclass',          # 目标函数：二分类
    'num_class': 3,
    'metric': 'multi_logloss',     # 评估指标
    'boosting_type': 'gbdt',        # 提升类型
    'n_estimators': 2000,           # 树的最大数量（一个较大的值，配合早停使用）
    'learning_rate': 0.01,          # 学习率
    'num_leaves': 31,               # 每棵树的叶子节点数
    'max_depth': -1,                # 树的最大深度, -1 表示无限制
    'seed': 42,                     # 随机种子，保证结果可复现
    'n_jobs': -1,                   # 使用所有可用的 CPU 核心
    'verbose': 0,                  # 抑制输出信息
    'colsample_bytree': 0.8,        # 构建每棵树时随机选择的特征比例
    'subsample': 0.8,               # 训练每棵树时使用的数据（行）比例
    'reg_alpha': 0.1,               # L1 正则化
    'reg_lambda': 0.1,              # L2 正则化
    'device': 'gpu'
}
best_params = {'learning_rate': 0.009054971028569354,
 'num_leaves': 25,
 'subsample': 0.7545176635949,
 'colsample_bytree': 0.6331222368386825,
 'reg_alpha': 8.847018473975617e-06,
 'reg_lambda': 1.990125011362269,
 'objective': 'multiclass',
 'num_class': 3,
 'metric': 'multi_logloss',
 'n_estimators': 2000,
 'seed': 42,
 'n_jobs': -1,
 'boosting_type': 'gbdt'}
params.update(best_params)

# 使用 **params 将字典解包成关键字参数来初始化分类器
lgbm = lgb.LGBMClassifier(**params)

# --- 模型训练与早停 ---
print("开始训练模型...")
lgbm.fit(X_train, y_train,
         eval_set=[(X_test, y_test)],        # 设置验证集用于早停
         eval_metric='multi_logloss',               # 在验证集上评估的指标
         callbacks=[lgb.early_stopping(200, verbose=True)]) # 如果验证集指标在 100 轮内没有改善，则停止训练

# --- 模型评估 ---
print("\n在测试集上进行评估...")
y_pred = lgbm.predict(X_test)
y_pred_proba = lgbm.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# --- 特征重要性 ---
# print("\n绘制特征重要性...")
# lgb.plot_importance(lgbm, max_num_features=len(feature_cols), figsize=(10, 6), importance_type='gain')
# plt.title('Feature Importance (Gain)')
# plt.show()


开始训练模型...
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[515]	valid_0's multi_logloss: 1.05917

在测试集上进行评估...
Accuracy: 0.4215

Classification Report:
              precision    recall  f1-score   support

           0       0.42      0.80      0.55      8865
           1       0.44      0.17      0.25      5399
           2       0.41      0.16      0.23      7743

    accuracy                           0.42     22007
   macro avg       0.42      0.38      0.34     22007
weighted avg       0.42      0.42      0.37     22007



In [ ]:
lgbm.booster_.save_model('lgbm_42.6.txt')

## optuna tuning

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import matplotlib.pyplot as plt
import optuna

# 假设您的数据在一个名为 all_in_one_file 的 DataFrame 中
# 并且 'label' 列包含 0, 1, 2 三个类别
all_in_one_file = pd.read_parquet('merged.parquet')

# --- 1. 数据准备 ---
emotion_cols = [
    'Emotion_Index_closing',
    'Emotion_Index_trading',
    'Emotion_Momentum_3d_closing',
    'Emotion_Momentum_3d_trading'
]
stock_cols = ['log_return', 'amplitude']
feature_cols = emotion_cols + stock_cols
target_col = 'Label'

X = all_in_one_file[feature_cols]
y = all_in_one_file[target_col]

# 划分训练集和测试集 (hold-out set)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

# --- 2. Optuna 目标函数 ---
def objective(trial):
    # 为超参数定义搜索空间
    params = {
        'objective': 'multiclass',
        'num_class': 3,
        'metric': 'multi_logloss',
        'boosting_type': 'gbdt',
        'n_estimators': 2000,
        'seed': 42,
        'n_jobs': -1,
        'verbose': -1,
        'device': 'gpu',
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': -1,
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
    }

    # 实例化模型
    model = lgb.LGBMClassifier(**params)

    # 训练模型
    model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
              eval_metric='multi_logloss',
              callbacks=[lgb.early_stopping(100, verbose=False)])

    # 在验证集上评估并返回准确率
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

# --- 3. 创建并运行 Optuna Study ---
# 我们希望最大化准确率，所以 direction='maximize'
study = optuna.create_study(direction='maximize')
print("开始使用 Optuna 进行 100 次参数调优...")
study.optimize(objective, n_trials=100)

# --- 4. 打印最佳结果 ---
print("\n调优完成！")
print("最佳试验:")
best_trial = study.best_trial
print(f"  价值 (Accuracy): {best_trial.value:.4f}")
print("  最佳参数: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# --- 5. 使用最佳参数训练最终模型 ---
print("\n使用最佳参数训练最终模型...")
best_params = best_trial.params
# 添加一些在调优中未设置的固定参数
best_params['objective'] = 'multiclass'
best_params['num_class'] = 3
best_params['metric'] = 'multi_logloss'
best_params['n_estimators'] = 2000 # 可以用一个较大的值，配合早停
best_params['seed'] = 42
best_params['n_jobs'] = -1
best_params['boosting_type'] = 'gbdt'


final_model = lgb.LGBMClassifier(**best_params)

# 在完整的训练集上训练，使用测试集作为早停的验证集
final_model.fit(X_train, y_train,
                eval_set=[(X_test, y_test)],
                eval_metric='multi_logloss',
                callbacks=[lgb.early_stopping(100, verbose=True)])

# --- 6. 评估最终模型 ---
print("\n在测试集上评估最终模型...")
y_pred_final = final_model.predict(X_test)

final_accuracy = accuracy_score(y_test, y_pred_final)
print(f"\nFinal Test Set Accuracy: {final_accuracy:.4f}")

print("\nFinal Model Classification Report:")
print(classification_report(y_test, y_pred_final))

# 绘制最终模型的特征重要性
lgb.plot_importance(final_model, max_num_features=len(feature_cols), figsize=(10, 6), importance_type='gain')
plt.title('Final Model Feature Importance (Gain)')
plt.show()


In [ ]:
best_trial.params

{'learning_rate': 0.009054971028569354,
 'num_leaves': 25,
 'subsample': 0.7545176635949,
 'colsample_bytree': 0.6331222368386825,
 'reg_alpha': 8.847018473975617e-06,
 'reg_lambda': 1.990125011362269,
 'objective': 'multiclass',
 'num_class': 3,
 'metric': 'multi_logloss',
 'n_estimators': 2000,
 'seed': 42,
 'n_jobs': -1,
 'boosting_type': 'gbdt'}

## Finetune on specific stock

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# ===== 1. 加载数据与模型 =====
# 加载和之前训练时相同的数据
all_in_one_file = pd.read_parquet('merged.parquet')
# 加载你保存的通用LGBM模型
general_model_path = 'lgbm_42.6.txt'

# 定义和之前训练时完全相同的特征
emotion_cols = [
    'Emotion_Index_closing',
    'Emotion_Index_trading',
    'Emotion_Momentum_3d_closing',
    'Emotion_Momentum_3d_trading'
]
stock_cols = ['log_return', 'amplitude']
feature_cols = emotion_cols + stock_cols
target_col = 'Label'

# ===== 2. 筛选特定股票数据 =====
stock_accs = {}
for stock_code_to_finetune in tqdm(all_in_one_file['StockCode'].unique()):
    df_stock = all_in_one_file[all_in_one_file['StockCode'] == stock_code_to_finetune].copy()
    df_stock = df_stock.sort_values('Date').reset_index(drop=True)

    if df_stock.empty:
        print(f"❌ 警告: 未找到股票代码 {stock_code_to_finetune} 的数据。")
    else:
        # ===== 3. 准备微调数据 =====
        X_stock = df_stock[feature_cols]
        y_stock = df_stock[target_col]

        # 按时间顺序划分微调用的训练集和测试集
        X_train_ft, X_test_ft, y_train_ft, y_test_ft = train_test_split(
            X_stock, y_stock, test_size=0.2, shuffle=False
        )
        # print(f"📊 微调数据集: {len(X_train_ft)} 训练样本, {len(X_test_ft)} 测试样本。")


        # ===== 4. 执行微调 =====
        # 使用和通用模型相同的参数配置，但可以降低学习率以进行更精细的调整
        # 我们这里沿用之前的 best_params
        best_params = {'learning_rate': 0.009054971028569354,
         'num_leaves': 25,
         'subsample': 0.7545176635949,
         'colsample_bytree': 0.6331222368386825,
         'reg_alpha': 8.847018473975617e-06,
         'reg_lambda': 1.990125011362269,
         'objective': 'multiclass',
         'num_class': 3,
         'metric': 'multi_logloss',
         'n_estimators': 2000,
         'seed': 42,
         'verbose': -1,
         'n_jobs': -1,
         'boosting_type': 'gbdt',
         'device': 'gpu'}

        # 降低学习率是一个常见的微调技巧，有助于模型在小数据集上更好地收敛
        finetune_params = best_params.copy()
        finetune_params['learning_rate'] = 0.005 # 例如，降到更低

        lgbm_finetune = lgb.LGBMClassifier(**finetune_params)

        # print("\n🚀 开始微调模型...")
        lgbm_finetune.fit(
            X_train_ft, y_train_ft,
            eval_set=[(X_test_ft, y_test_ft)],
            eval_metric='multi_logloss',
            callbacks=[lgb.early_stopping(10, verbose=False)],
            init_model=general_model_path  # <-- 微调的关键：加载预训练模型
        )

        # ===== 5. 评估微调后的模型 =====
        # print("\n🔍 评估微调后的模型在专属测试集上的表现...")
        y_pred_ft = lgbm_finetune.predict(X_test_ft)
        accuracy_ft = accuracy_score(y_test_ft, y_pred_ft)
        stock_accs[stock_code_to_finetune] = {
            "acc": accuracy_ft,
            "v_counts": y_test_ft.value_counts()
        }

        # print(f"\nFinetuned Model Accuracy: {accuracy_ft:.4f}")
        # print("\nFinetuned Model Classification Report:")
        # print(classification_report(y_test_ft, y_pred_ft))


100%|██████████| 230/230 [01:28<00:00,  2.61it/s]


In [ ]:
stock_accs

{'000001': {'acc': 0.3673469387755102,
  'v_counts': Label
  0    17
  1    16
  2    16
  Name: count, dtype: int64},
 '000002': {'acc': 0.5102040816326531,
  'v_counts': Label
  0    23
  2    16
  1    10
  Name: count, dtype: int64},
 '000063': {'acc': 0.4489795918367347,
  'v_counts': Label
  2    21
  0    20
  1     8
  Name: count, dtype: int64},
 '000301': {'acc': 0.40816326530612246,
  'v_counts': Label
  2    22
  0    21
  1     6
  Name: count, dtype: int64},
 '000333': {'acc': 0.3673469387755102,
  'v_counts': Label
  0    17
  2    17
  1    15
  Name: count, dtype: int64},
 '000338': {'acc': 0.32653061224489793,
  'v_counts': Label
  0    18
  2    18
  1    13
  Name: count, dtype: int64},
 '000568': {'acc': 0.46938775510204084,
  'v_counts': Label
  0    23
  1    14
  2    12
  Name: count, dtype: int64},
 '000596': {'acc': 0.5102040816326531,
  'v_counts': Label
  0    23
  2    18
  1     8
  Name: count, dtype: int64},
 '000630': {'acc': 0.40816326530612246,
  'v_

In [ ]:
for code, v in sorted(stock_accs.items(), key=lambda item: item[1]['acc'], reverse=True):
    print(f"{code}: {v['acc']} {max(v['v_counts']) / sum(v['v_counts'])}")


002920: 0.6326530612244898 0.5306122448979592
300782: 0.5918367346938775 0.5510204081632653
600460: 0.5918367346938775 0.42857142857142855
600760: 0.5918367346938775 0.4897959183673469
300015: 0.5714285714285714 0.5102040816326531
601698: 0.5714285714285714 0.46938775510204084
002230: 0.5510204081632653 0.42857142857142855
300347: 0.5510204081632653 0.5510204081632653
300394: 0.5510204081632653 0.4897959183673469
300502: 0.5510204081632653 0.4489795918367347
601111: 0.5510204081632653 0.42857142857142855
601865: 0.5510204081632653 0.42857142857142855
603806: 0.5510204081632653 0.5306122448979592
688082: 0.5510204081632653 0.46938775510204084
688472: 0.5510204081632653 0.5102040816326531
000977: 0.5306122448979592 0.4489795918367347
002027: 0.5306122448979592 0.4489795918367347
002142: 0.5306122448979592 0.42857142857142855
002180: 0.5306122448979592 0.4489795918367347
002236: 0.5306122448979592 0.46938775510204084
002371: 0.5306122448979592 0.4489795918367347
300628: 0.5306122448979592

In [ ]:
max(all_in_one_file[all_in_one_file['StockCode'] == '600089']['Label'].value_counts())

99

PatchTST

In [ ]:
# ============================================
# 🚀 Loop fine-tuning across all StockCodes & rank top-10 by metrics
#   NOTE* - RUN ON GOOGLE COLAB
# ============================================
!pip -q install --upgrade pip
!pip -q install "transformers>=4.46.0" "accelerate>=0.34.0" pyarrow tqdm scikit-learn pandas numpy

import re, os, math, pandas as pd, numpy as np, torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
from transformers import PatchTSTModel
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# ============================================
# 📂 Paths / globals
# ============================================
from google.colab import drive
drive.mount('/content/drive')

PATH_PARQUET   = "/content/drive/MyDrive/merged.parquet"
BASE_CKPT      = "/content/drive/MyDrive/best_model.pt"        # previously trained general ckpt
OUT_DIR        = "/content/drive/MyDrive/ft_per_code"          # where per-code ckpts & CSV go
os.makedirs(OUT_DIR, exist_ok=True)

# ============================================
# ⚙️ Common hyperparams for per-code FT
# ============================================
window_size     = 90         # try 60/30/90 if you'd like to sweep
train_ratio     = 0.85
batch_size      = 64
num_classes     = 3
num_epochs      = 15         # keep modest for 230 codes; early stopping included
patience        = 3          # early stop on val loss
warmup_epochs   = 2

# Optim (head-only to start; set UNFREEZE_EPOCH to lightly unfreeze later)
lr_head         = 5e-4
lr_backbone     = 1e-6
weight_decay    = 5e-4
grad_clip       = 1.0
UNFREEZE_EPOCH  = None       # e.g., 6 to unfreeze last block lightly

label_smoothing = 0.10
USE_CLASS_WEIGHTS = True

# Granite PatchTST dims
BACKBONE_ID       = "ibm-granite/granite-timeseries-patchtst"
proj_dim          = 128
hidden_fused      = 512
granite_channels  = 7

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# ============================================
# 🧩 Columns (emotion + stock)
# ============================================
emotion_cols = [
    'Emotion_Index_closing',
    'Emotion_Index_trading',
    'Emotion_Momentum_3d_closing',
    'Emotion_Momentum_3d_trading'
]
stock_cols = ['log_return', 'amplitude']
feature_cols = emotion_cols + stock_cols
LABEL_COL = "Label"
CODE_COL  = "StockCode"
DATE_COL  = "Date"

# ============================================
# 🧠 Model & helpers (same architecture as before)
# ============================================
class Adapter(nn.Module):
    def __init__(self, hidden_size, bottleneck=64, dropout=0.05, scale=1.0):
        super().__init__()
        self.ln = nn.LayerNorm(hidden_size)
        self.down = nn.Linear(hidden_size, bottleneck)
        self.up   = nn.Linear(bottleneck, hidden_size)
        self.dropout = nn.Dropout(dropout)
        self.scale = scale
        nn.init.kaiming_uniform_(self.down.weight, a=math.sqrt(5))
        nn.init.zeros_(self.down.bias); nn.init.zeros_(self.up.weight); nn.init.zeros_(self.up.bias)
    def forward(self, x):
        z = self.ln(x); z = torch.relu(self.down(z)); z = self.dropout(z); z = self.up(z)
        return x + self.scale * z

def right_pad_to_context(x, context_len, pad_value=0.0):
    B,T,C = x.shape
    if T > context_len:
        x = x[:, -context_len:, :]; T = context_len
    pad_len = context_len - T
    if pad_len > 0:
        pad = x.new_full((B,pad_len,C), pad_value)
        x = torch.cat([pad, x], dim=1)
        mask = torch.cat([torch.zeros(B,pad_len,C, device=x.device),
                          torch.ones(B,T,C, device=x.device)], dim=1)
    else:
        mask = torch.ones(B,T,C, device=x.device)
    return x, mask

def sanitize_logits(logits, num_classes):
    if logits.dim()==3:
        if logits.size(-1)==num_classes: logits = logits.mean(dim=1)
        elif logits.size(1)==num_classes: logits = logits.mean(dim=2)
        else: logits = logits.mean(dim=1)
    return logits

class EmoStockToGranite(nn.Module):
    def __init__(self, stock_dim=2, emo_dim=4,
                 proj_dim=128, fused_dim=512, granite_channels=7,
                 num_classes=3, backbone_id=BACKBONE_ID):
        super().__init__()
        self.stock_proj = nn.Sequential(nn.Linear(stock_dim, proj_dim), nn.ReLU(), nn.LayerNorm(proj_dim))
        self.emo_proj   = nn.Sequential(nn.LayerNorm(emo_dim), nn.Linear(emo_dim, proj_dim), nn.ReLU(), nn.LayerNorm(proj_dim))
        self.fuse       = nn.Sequential(nn.Linear(proj_dim*2, fused_dim), nn.ReLU(), nn.LayerNorm(fused_dim))
        self.channel_aligner = nn.Linear(fused_dim, granite_channels)

        self.backbone = PatchTSTModel.from_pretrained(backbone_id)
        self.context_len = int(self.backbone.config.context_length)
        d_model = int(self.backbone.config.d_model)

        for p in self.backbone.parameters():
            p.requires_grad = False  # head warmup by default

        self.adapter  = Adapter(d_model, bottleneck=64, dropout=0.05)
        self.cls_head = nn.Sequential(nn.Linear(d_model, d_model//2), nn.ReLU(), nn.Dropout(0.1), nn.Linear(d_model//2, num_classes))

    def forward(self, stock_seq, emo_seq):
        s = self.stock_proj(stock_seq)                      # [B,T,P]
        e = self.emo_proj(emo_seq)                          # [B,T,P]
        fused = self.fuse(torch.cat([s,e], dim=-1))         # [B,T,F]
        pv = self.channel_aligner(fused)                    # [B,T,C]
        pv, obs_mask = right_pad_to_context(pv, self.context_len)
        out = self.backbone(past_values=pv, past_observed_mask=obs_mask.to(dtype=torch.bool, device=pv.device), output_hidden_states=True)
        hs = self.adapter(out.last_hidden_state)            # [B,context_len,d_model]
        pooled = hs.mean(dim=1)                             # mean pooling
        return self.cls_head(pooled)                        # [B,num_classes]

def unfreeze_top_blocks(mdl, n_blocks=1):
    names = [n for n,_ in mdl.backbone.named_parameters()]
    layer_idx = []
    for n in names:
        m = re.search(r"layers\.(\d+)", n)
        if m: layer_idx.append(int(m.group(1)))
    changed = 0
    if layer_idx:
        max_idx = max(layer_idx)
        target = set(range(max_idx - (n_blocks-1), max_idx + 1))
        for n,p in mdl.backbone.named_parameters():
            m = re.search(r"layers\.(\d+)", n)
            if m and int(m.group(1)) in target:
                p.requires_grad = True; changed += 1
    else:
        for p in mdl.backbone.parameters(): p.requires_grad = True
        changed = -1
    return changed

def decay_filter(n, p):
    return p.requires_grad and (p.dim() > 1) and ("bias" not in n) and ("LayerNorm" not in n) and ("layer_norm" not in n)

def build_param_groups(model, lr_head, lr_backbone, wd):
    backbone_decay, backbone_nodecay, head_decay, head_nodecay = [], [], [], []
    for n,p in model.named_parameters():
        if not p.requires_grad: continue
        is_backbone = n.startswith("backbone.")
        if decay_filter(n,p):
            (backbone_decay if is_backbone else head_decay).append(p)
        else:
            (backbone_nodecay if is_backbone else head_nodecay).append(p)
    groups = []
    if backbone_decay:   groups.append({"params": backbone_decay,   "lr": lr_backbone, "weight_decay": wd})
    if backbone_nodecay: groups.append({"params": backbone_nodecay, "lr": lr_backbone, "weight_decay": 0.0})
    if head_decay:       groups.append({"params": head_decay,       "lr": lr_head,     "weight_decay": wd})
    if head_nodecay:     groups.append({"params": head_nodecay,     "lr": lr_head,     "weight_decay": 0.0})
    return groups

def make_scheduler(optimizer, num_epochs, warmup_epochs):
    def lr_lambda(epoch):
        if epoch < warmup_epochs:
            return (epoch + 1) / max(1, warmup_epochs)
        progress = (epoch - warmup_epochs) / max(1, (num_epochs - warmup_epochs))
        return 0.1 + 0.9 * 0.5 * (1 + math.cos(math.pi * progress))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lr_lambda)

# ============================================
# 🧼 Per-code data prep
# ============================================
def prepare_code_windows(df_code, window_size, train_ratio):
    # basic cleaning (single code)
    df_code = df_code.sort_values([CODE_COL, DATE_COL]).reset_index(drop=True)
    # replace inf, ffill/bfill
    df_code[feature_cols] = df_code[feature_cols].replace([np.inf, -np.inf], np.nan)
    df_code[feature_cols] = df_code[feature_cols].ffill().bfill()
    # fill remaining with medians
    df_code[feature_cols] = df_code[feature_cols].fillna(df_code[feature_cols].median(numeric_only=True))
    # robust clip & zscore
    lo = df_code[feature_cols].quantile(0.01); hi = df_code[feature_cols].quantile(0.99)
    df_code[feature_cols] = df_code[feature_cols].clip(lower=lo, upper=hi, axis=1)
    mu = df_code[feature_cols].mean(); sd = df_code[feature_cols].std(ddof=0).replace(0, 1.0)
    df_code[feature_cols] = (df_code[feature_cols] - mu) / sd

    # build windows
    samples = []
    emo_np = df_code[emotion_cols].to_numpy(dtype=np.float32, copy=True)
    stk_np = df_code[stock_cols].to_numpy(dtype=np.float32, copy=True)
    emo_np[~np.isfinite(emo_np)] = np.nan
    stk_np[~np.isfinite(stk_np)] = np.nan
    emo = torch.from_numpy(np.nan_to_num(emo_np, nan=0.0, posinf=0.0, neginf=0.0))
    stk = torch.from_numpy(np.nan_to_num(stk_np, nan=0.0, posinf=0.0, neginf=0.0))
    y   = torch.tensor(df_code[LABEL_COL].values, dtype=torch.int64)
    dt  = df_code[DATE_COL].values

    for i in range(len(df_code) - window_size):
        emos  = emo[i:i+window_size]
        stks  = stk[i:i+window_size]
        label = y[i + window_size]
        if not (torch.isfinite(emos).all() and torch.isfinite(stks).all()): continue
        samples.append({"emotion": emos, "stock": stks, "label": label, "end_date": dt[i+window_size]})

    if len(samples) == 0:
        return None, None, None, None

    dates = sorted(pd.Series([s["end_date"] for s in samples]).unique())
    split_date = dates[int(len(dates) * train_ratio)]
    train_samples = [s for s in samples if s["end_date"] <= split_date]
    val_samples   = [s for s in samples if s["end_date"] >  split_date]

    class WindowDataset(Dataset):
        def __init__(self, samples): self.samples = samples
        def __len__(self): return len(self.samples)
        def __getitem__(self, idx): return self.samples[idx]
    def collate_batch(batch):
        return {
            "emotion": torch.stack([b["emotion"] for b in batch], dim=0),
            "stock":   torch.stack([b["stock"]   for b in batch], dim=0),
            "label":   torch.stack([b["label"]   for b in batch], dim=0),
            "end_date": [pd.Timestamp(b["end_date"]).to_pydatetime() for b in batch],
        }

    train_loader = DataLoader(WindowDataset(train_samples), batch_size=batch_size, shuffle=True,  collate_fn=collate_batch, pin_memory=torch.cuda.is_available())
    val_loader   = DataLoader(WindowDataset(val_samples),   batch_size=batch_size, shuffle=False, collate_fn=collate_batch, pin_memory=torch.cuda.is_available())

    train_labels = [int(s["label"]) for s in train_samples]
    val_labels   = [int(s["label"]) for s in val_samples]
    return train_loader, val_loader, train_labels, val_labels

# ============================================
# 🏋️ Finetune single code & return metrics
# ============================================
def finetune_one_code(code, df, save_ckpt=True):
    df_code = df[df[CODE_COL].astype(str) == code].copy()
    if len(df_code) <= window_size + 10:
        return {"StockCode": code, "support": 0, "accuracy": np.nan,
                "weighted_precision": np.nan, "weighted_recall": np.nan, "weighted_f1": np.nan,
                "best_val_acc": np.nan, "best_val_loss": np.nan, "notes": "too_few_rows"}

    tl, vl, train_labels, val_labels = prepare_code_windows(df_code, window_size, train_ratio)
    if tl is None or len(val_labels) == 0:
        return {"StockCode": code, "support": 0, "accuracy": np.nan,
                "weighted_precision": np.nan, "weighted_recall": np.nan, "weighted_f1": np.nan,
                "best_val_acc": np.nan, "best_val_loss": np.nan, "notes": "no_windows"}

    # Build model & load base checkpoint
    model = EmoStockToGranite(
        stock_dim=len(stock_cols),
        emo_dim=len(emotion_cols),
        proj_dim=proj_dim,
        fused_dim=hidden_fused,
        granite_channels=granite_channels,
        num_classes=num_classes,
        backbone_id=BACKBONE_ID
    ).to(device)
    try:
        model.load_state_dict(torch.load(BASE_CKPT, map_location=device), strict=False)
    except Exception as e:
        return {"StockCode": code, "support": len(val_labels), "accuracy": np.nan,
                "weighted_precision": np.nan, "weighted_recall": np.nan, "weighted_f1": np.nan,
                "best_val_acc": np.nan, "best_val_loss": np.nan, "notes": f"load_fail:{e}"}

    # Loss & weights per this code
    if USE_CLASS_WEIGHTS:
        counts = np.bincount(np.array(train_labels, dtype=int), minlength=num_classes)
        class_weights = 1.0 / np.clip(counts, 1, None)
        class_weights = class_weights * (num_classes / class_weights.sum())
        class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32, device=device)
    else:
        class_weights_tensor = None
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor, label_smoothing=label_smoothing)

    # Head-only optim
    def build_optim(m):
        groups = build_param_groups(m, lr_head, lr_backbone, weight_decay)
        opt = optim.AdamW(groups)
        sch = make_scheduler(opt, num_epochs=num_epochs, warmup_epochs=warmup_epochs)
        return opt, sch
    optimizer, scheduler = build_optim(model)

    # Optional partial unfreeze later
    def maybe_unfreeze(epoch):
        nonlocal optimizer, scheduler
        if UNFREEZE_EPOCH is not None and epoch == UNFREEZE_EPOCH:
            _ = unfreeze_top_blocks(model, n_blocks=1)
            optimizer, scheduler = build_optim(model)

    # Eval
    def evaluate(model_for_eval, vl):
        model_for_eval.eval()
        val_loss = 0.0; val_correct = 0; val_total = 0
        all_preds = []; all_true = []
        with torch.no_grad():
            for batch in vl:
                emo = batch["emotion"].to(device)
                stk = batch["stock"].to(device)
                y   = batch["label"].to(device)
                logits = sanitize_logits(model_for_eval(stk, emo), num_classes)
                loss = criterion(logits, y)
                val_loss += loss.item() * y.size(0)
                preds = logits.argmax(1)
                val_correct += (preds == y).sum().item()
                val_total += y.size(0)
                all_preds.append(preds.detach().cpu().numpy())
                all_true.append(y.detach().cpu().numpy())
        acc = val_correct / max(1,val_total)
        return (val_loss / max(1,val_total), acc, np.concatenate(all_true), np.concatenate(all_preds))

    # Train with early stopping on val loss
    best_val = float("inf")
    best_acc = 0.0
    epochs_no_improve = 0
    ckpt_path = os.path.join(OUT_DIR, f"best_{code}.pt")

    for epoch in range(1, num_epochs+1):
        maybe_unfreeze(epoch)
        model.train()
        train_loss = 0.0; train_correct = 0; train_total = 0
        for batch in tl:
            emo = batch["emotion"].to(device)
            stk = batch["stock"].to(device)
            y   = batch["label"].to(device)
            optimizer.zero_grad()
            logits = sanitize_logits(model(stk, emo), num_classes)
            loss = criterion(logits, y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()
            train_loss += loss.item() * y.size(0)
            train_correct += (logits.argmax(1) == y).sum().item()
            train_total += y.size(0)
        scheduler.step()

        val_loss, val_acc, y_true, y_pred = evaluate(model, vl)
        if val_loss < best_val - 1e-4:
            best_val = val_loss
            best_acc = val_acc
            epochs_no_improve = 0
            if save_ckpt:
                torch.save(model.state_dict(), ckpt_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                break

    # Load best & compute final metrics
    if save_ckpt and os.path.exists(ckpt_path):
        model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=False)
    val_loss, val_acc, y_true, y_pred = evaluate(model, vl)

    # weighted precision/recall/f1 via sklearn
    try:
        report = classification_report(y_true, y_pred, digits=4, output_dict=True, zero_division=0)
        wP = report["weighted avg"]["precision"]
        wR = report["weighted avg"]["recall"]
        wF = report["weighted avg"]["f1-score"]
        support = int(report["accuracy"] * len(y_true))  # not right: fix below
        support = int(len(y_true))                      # support = # val samples
    except Exception:
        wP = wR = wF = np.nan
        support = len(y_true)

    return {
        "StockCode": code,
        "support": support,
        "accuracy": float(val_acc),
        "weighted_precision": float(wP),
        "weighted_recall": float(wR),
        "weighted_f1": float(wF),
        "best_val_acc": float(best_acc),
        "best_val_loss": float(best_val),
        "ckpt_path": (ckpt_path if save_ckpt else ""),
        "notes": ""
    }

# ============================================
# 📥 Load parquet & get all codes
# ============================================
df_all = pd.read_parquet(PATH_PARQUET)
for col in [CODE_COL, DATE_COL, LABEL_COL] + feature_cols:
    if col not in df_all.columns:
        raise ValueError(f"Missing column: {col}")
df_all[LABEL_COL] = df_all[LABEL_COL].astype("int64")

codes = sorted(df_all[CODE_COL].dropna().astype(str).unique().tolist())
print(f"Found {len(codes)} unique StockCodes.")

# If you want to test on a subset first, uncomment:
# codes = codes[:10]

# ============================================
# 🔁 Loop all codes, collect metrics
# ============================================
all_rows = []
pbar = tqdm(codes, desc="Per-code finetune", ncols=100)
for code in pbar:
    pbar.set_postfix_str(code)
    row = finetune_one_code(code, df_all, save_ckpt=True)
    all_rows.append(row)

metrics_df = pd.DataFrame(all_rows)
metrics_csv = os.path.join(OUT_DIR, "per_code_metrics.csv")
metrics_df.to_csv(metrics_csv, index=False)
print(f"\n✅ Saved per-code metrics to: {metrics_csv}")

# ============================================
# 🏆 Show Top-10 by requested metrics
# ============================================
def show_top10(df, col, higher_is_better=True):
    df2 = df.copy()
    df2 = df2.dropna(subset=[col])
    df2 = df2.sort_values(col, ascending=not higher_is_better)
    return df2[["StockCode", col, "support", "accuracy", "weighted_precision", "weighted_recall", "weighted_f1"]].head(10)

print("\n=== Top-10 by Accuracy ===")
print(show_top10(metrics_df, "accuracy", True))

print("\n=== Top-10 by Support ===")
print(show_top10(metrics_df, "support", True))

print("\n=== Top-10 by Weighted Precision ===")
print(show_top10(metrics_df, "weighted_precision", True))

print("\n=== Top-10 by Weighted Recall ===")
print(show_top10(metrics_df, "weighted_recall", True))

print("\n=== Top-10 by Weighted F1 ===")
print(show_top10(metrics_df, "weighted_f1", True))

print(f"\n📁 Checkpoints & CSV are in: {OUT_DIR}")
